In [1]:
import tensorflow as tf
import os
import numpy as np
from utils import *
import datetime
from PIL import Image

In [2]:
!export CUDA_VISIBLE_DEVICES=0,1

In [3]:
mirrored_strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [5]:
with mirrored_strategy.scope():
    batch_size = 16
    dropout = 0.7
    ls = 0.0
    lr = 1e-4
    epochs = 1000

In [8]:
with mirrored_strategy.scope():
    ROOT_DIR = get_git_root(os.getcwd())
    train_path = os.path.join(ROOT_DIR, 'datasets', 'veri-wild', 'mini', 'train')
    val_path = os.path.join(ROOT_DIR, 'datasets', 'veri-wild', 'mini', 'val')
    train_gen = tf.keras.preprocessing.image.ImageDataGenerator()
    val_gen = tf.keras.preprocessing.image.ImageDataGenerator()

    train_gen = val_gen.flow_from_directory(train_path, target_size=(299,299),
                                              batch_size=batch_size)
    val_gen = val_gen.flow_from_directory(val_path, target_size=(299,299),
                                            batch_size=batch_size)

Found 3132 images belonging to 1000 classes.
Found 3253 images belonging to 1000 classes.


In [33]:
with mirrored_strategy.scope():
    model = tf.keras.models.load_model(os.path.join(ROOT_DIR, 'models', 'EfB3-full-299'))

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 299, 299, 3)]     0         
_________________________________________________________________
efficientnetb3 (Functional)  (None, 1536)              10783535  
_________________________________________________________________
Flatten_1 (Flatten)          (None, 1536)              0         
Total params: 10,783,535
Trainable params: 10,696,232
Non-trainable params: 87,303
_________________________________________________________________


In [35]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 299, 299, 3)]     0         
_________________________________________________________________
efficientnetb3 (Functional)  (None, 1536)              10783535  
_________________________________________________________________
Flatten_1 (Flatten)          (None, 1536)              0         
_________________________________________________________________
Dense_1 (Dense)              (None, 2048)              3147776   
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
Dense_2 (Dense)              (None, 2048)              4196352   
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0     

In [42]:
with mirrored_strategy.scope():
    encoder = tf.keras.models.Model(model.input, model.get_layer('Dense_3').output)
    input_layer = tf.keras.layers.Input(shape=train_gen.next()[0][0].shape)
    output = encoder(input_layer)
    output = Dense(1000, activation='softmax', name='Dense_output')(output)
    veri_model = tf.keras.models.Model(input_layer, output)
    
    for each_layer in veri_model.layers[:-1]:
        each_layer.trainable = False
    
    veri_model.summary()

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 299, 299, 3)]     0         
_________________________________________________________________
model_16 (Functional)        (None, 1000)              20176663  
_________________________________________________________________
Dense_output (Dense)         (None, 1000)              1001000   
Total params: 21,177,663
Trainable params: 1,001,000
Non-trainable params: 20,176,663
_________________________________________________________________


In [47]:
with mirrored_strategy.scope():
    loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=ls)
    metrics = 'categorical_accuracy'
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    veri_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    veri_model.fit(train_gen, validation_data=val_gen, epochs=epochs, verbose=1)

Epoch 1/1000
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1


196/196 [==============================] - 26s 79ms/step - loss: 4.0450 - categorical_accuracy: 0.2427 - val_loss: 5.0186 - val_categorical_accuracy: 0.1721
Epoch 2/1000
196/196 [==============================] - 10s 53ms/step - loss: 3.7814 - categorical_accuracy: 0.2599 - val_loss: 4.8996 - val_categorical_accuracy: 0.1943
Epoch 3/1000
196/196 [==============================] - 10s 53ms/step - loss: 3.5417 - categorical_accuracy: 0.2784 - val_loss: 4.8082 - val_categorical_accuracy: 0.2164
Epoch 4/1000
196/196 [==============================] - 10s 53ms/step - loss: 3.4273 - categorical_accuracy: 0.3126 - val_loss: 4.7371 - val_categorical_accuracy: 0.2284
Epoch 5/1000
196/196 [==============================] - 10s 53ms/step - loss: 3.2008 - categorical_accuracy: 0.3630 - val_loss: 4.6816 - val_categorical_accuracy: 0.2361
Epoch 6/1000
196/196 [==============================] - 10s 52ms/step - loss: 3.1112 - categorical_accuracy: 0.3785 - val_loss: 4.6302 - val_categorical_accuracy: 

KeyboardInterrupt: 

In [18]:
model.save_weights(os.path.join(ROOT_DIR, 'trainings', 'models', 'tmp'))

In [6]:
with mirrored_strategy.scope():
    input_layer = tf.keras.layers.Input(shape=train_gen.next()[0][0].shape)
    model2 = tf.keras.applications.EfficientNetB7(include_top=False,
                                                 input_tensor=input_layer)
    for each_layer in model2.layers:
        each_layer.trainable = False

    output = model2(input_layer)
    output = AveragePooling2D((5,5), name='avg_pool')(output)
    output = Dropout(dropout)(output)
    output = Flatten(name='Flatten_1')(output)
    output = Dense(1024, activation='relu', name='Dense_preoutput')(output)
    output = Dropout(dropout)(output)
    output = Dense(30671, activation='softmax', name='Dense_output')(output)

    model2 = tf.keras.models.Model(input_layer, output)

    model2.load_weights(os.path.join(ROOT_DIR, 'trainings', 'models', 'tmp'))
    model2.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
with mirrored_strategy.scope():
    model2.fit(train_gen,
              epochs=epochs)

Epoch 1/1000
INFO:tensorflow:batch_all_reduce: 4 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 4 all-reduces with algorithm = nccl, num_packs = 1
 618/1918 [========>.....................] - ETA: 1:55 - loss: 3.2911 - categorical_accuracy: 0.4939

In [20]:
ROOT_DIR = get_git_root(os.getcwd())
train_path = os.path.join(ROOT_DIR, 'veri-wild', 'images', 'train')
val_path = os.path.join(ROOT_DIR, 'veri-wild', 'images', 'val')
labels = os.listdir(train_path)
print(np.array(labels).shape)
labels += os.listdir(val_path)
print(np.array(labels).shape)
labels = list(set(labels))
print(np.array(labels).shape)

(30671,)
(40672,)
(40671,)


In [15]:
np.array(labels).shape

(40671,)

In [19]:
labels.sort()
labels[-1]

'40671'